In [1]:
import pandas as pd
import json
from openai import OpenAI

In [2]:
import time
import os

In [3]:
client = OpenAI()


In [5]:
assistant = client.beta.assistants.create(
    name = "KnowledgeBased Chatbot",
    instructions= "Use provided knowledge bases to answer questions.",
    tools = [{"type":"retrieval"}, {"type":"code_interpreter"}],
    model = "gpt-3.5-turbo-1106"
)


numerical_file = client.files.create(
  file=open("all_ai_tool.csv", "rb"),
  purpose='assistants'
)

text_file = client.files.create(
  file=open("AI.txt", "rb"),
  purpose='assistants'
)

assistant = client.beta.assistants.create(
    name = "KnowledgeBased Chatbot",
    instructions= "Use provided knowledge bases to answer questions.",
    tools = [{"type":"retrieval"}, {"type":"code_interpreter"}],
    model = "gpt-3.5-turbo-1106",
    file_ids=[numerical_file.id, text_file.id]
)

In [6]:
ai_query = input("Ask a question about artificial intelligence: ")


In [15]:
def answer_question_(question):
    thread = client.beta.threads.create()

    client.beta.threads.messages.create(
        thread_id= thread.id,
        role = "user",
        content = question)
    
    run = client.beta.threads.runs.create(
        thread_id= thread.id,
        assistant_id= assistant.id,
    )


    while run.status != "completed":
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    
    
    messages = client.beta.threads.messages.list(
    thread_id= thread.id)
    


    for message in messages:
        if message.role == 'assistant':
            if hasattr(message.content, 'text'):
                print(message.content)

In [20]:

import time

def answer_question(question):
    try:
        # Create a new thread for the conversation
        thread = client.beta.threads.create()

        # Add the user's query as a message in the thread
        client.beta.threads.messages.create(
            thread_id=thread.id,
            role="user",
            content=question)
        
        # Create a run to process the query with the assistant
        run = client.beta.threads.runs.create(
            thread_id=thread.id,
            assistant_id=assistant.id,
        )

        # Wait for the run to complete
        while run.status != "completed":
            time.sleep(1)
            run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
            print("Run Status:", run.status)  # Debug print

        # Retrieve all messages from the thread
        messages = client.beta.threads.messages.list(thread_id=thread.id)

        # Iterate through the messages and print the assistant's response
        for message in messages:
            if message.role == 'assistant':
                # Check the content structure and access the text value
                if hasattr(message.content, 'text') and hasattr(message.content.text, 'value'):
                    # Print only the text value
                    print(message.content.text.value)
                else:
                    print("Unexpected content structure or empty response:", message.content)
    except Exception as e:
        print("An error occurred:", e)

# Example usage
ai_query = input("Ask a question about artificial intelligence: ")
answer_question(ai_query)


Run Status: in_progress
Run Status: in_progress
Run Status: completed
Unexpected content structure or empty response: [MessageContentText(text=Text(annotations=[], value='Chatbot GPT (Generative Pre-trained Transformer) is a type of conversational AI model developed by OpenAI. It is based on the transformer architecture and is trained on a diverse range of internet text, allowing it to generate human-like responses to input text. Chatbot GPT is designed to understand and respond to human language, making it suitable for a wide range of conversational applications, such as customer support, language translation, and personal assistants.\n\nDo you have a specific document related to Chatbot GPT that you would like me to review?'), type='text')]
